# Package Imports

In [2]:
# Data Processing

import data_processing_code.processing_functions as pf

from datetime import datetime as dt
# import json
import numpy as np
import pandas as pd
# import urllib.request

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from statsmodels.tsa.seasonal import STL


import warnings
warnings.filterwarnings("ignore")

# Data Acquisition 

Our data is sourced through Open-Meteo's Historical Weather API.

## Selections for Data Download 

**General Details:**

- Data Source: https://open-meteo.com/en/docs/historical-weather-api

- API Call: https://archive-api.open-meteo.com/v1/era5?latitude=43.70011&longitude=-79.4163&start_date=1984-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,cloud_cover,wind_speed_10m,sunshine_duration&timezone=America%2FNew_York

https://archive-api.open-meteo.com/v1/era5?latitude=43.70011&longitude=-79.4163&start_date=1984-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,cloud_cover,wind_speed_10m,sunshine_duration,shortwave_radiation&timezone=America%2FNew_York

Per the documentation, we specify era5 in the API call, in order to ensure data consistency and to prevent unintentional alterations that may have arised from the adoption of different weather model upgrades. 


**Toronto Details:**

- Latitude: 43.70011  
- Longitude: -79.4163  
- Timezone: America/New_York  
- Start Date: 1984-01-01  
- End Date: 2023-12-31  

We will primarily focus our analyses on data between 1994 and 2023; however, we are downloading an additional decade's worth, in case it becomes necessary. 


Although Daily Weather Variables are available for selection, we opt for Hourly Variables instead, so that we can include Cloud Cover information in our analyses. 


**Hourly Weather Variables:**
- Temperature (2 m)
- Relative Humidity (2 m)
- Dew Point (2 m)
- Precipitation (rain + snow)
- Cloud Cover Total
- Wind Speed (10 m)

**Additional Variables:**
- Sunshine Duration

**Solar Radiation Variables:**
- Shortwave Solar Radition GHI

Although this variable will not be used in the experiments, we download it to assess the strength of its relationship to sunshine duration. 

**Settings:**

- Temperature Unit: Celsius  
- Wind Speed Unit: Km/h  
- Precipitation Unit: Millimeter  
- Timeformat: ISO 8601 (e.g. 2022-12-31)  


In [5]:
# Retrieve data from Open Meteo
file_path = 'gdrive/MyDrive/MRP/data/'
file_name = 'era5_data_toronto_retrieved_2024-07-12.json'
api_call = 'https://archive-api.open-meteo.com/v1/archive?latitude=43.70011&longitude=-79.4163&start_date=1984-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,cloud_cover,wind_speed_10m,sunshine_duration&timezone=America%2FNew_York'
download_data(api_call, file_path, file_name)
